第三章 案例一 —— 为什么销售额会减少

In [ ]:
import os
import pandas as pd
import numpy as np

读入CSV文件

In [ ]:
filepath = os.getcwd() + '/datasets/' 

In [ ]:
## 包含各个日期里使用该应用的全部用户的ID
dau = pd.read_csv(filepath + 'section3-dau.csv') 
dau.head()

In [ ]:
## 保存有消费行为的用户每日所消费的金额数据
dpu = pd.read_csv(filepath + 'section3-dpu.csv')
dpu.head()

In [ ]:
## 保存各个用户是在何年何月何日首次玩这个应用的数据
install = pd.read_csv(filepath + 'section3-install.csv')
install.head()

合并数据集

In [ ]:
dau_install = dau.merge(install, how='inner', on=['user_id', 'app_name'])
dau_install.head()

In [ ]:
dau_install_payment = dau_install.merge(dpu, how='left', on=['log_date', 'app_name', 'user_id'])
dau_install_payment.head()

In [ ]:
dau_install_payment.loc[dau_install_payment.payment.notnull()].head()

In [ ]:
## 将未消费的记录的消费额设置为0
dau_install_payment.payment.fillna(0, inplace=True)
dau_install_payment.head()

按月统计

In [ ]:
## 增加一列表示月份
dau_install_payment['log_month'] = dau_install_payment['log_date'].apply(lambda x: x[:7])
dau_install_payment['install_month'] = dau_install_payment['install_date'].apply(lambda x: x[:7])
dau_install_payment.head()

In [ ]:
mau_payment = dau_install_payment.groupby(['user_id', 'log_month', 'install_month'])['payment'].sum().reset_index()
mau_payment.head()

增加属性来区分新用户和已有用户

In [ ]:
# mau_payment['user_type'] = None
# mau_payment.loc[mau_payment['install_month'] == mau_payment['log_month'], 'user_type'] = 'install'
# mau_payment.loc[mau_payment['install_month'] != mau_payment['log_month'], 'user_type'] = 'existing'

mau_payment['user_type'] = np.where((mau_payment['install_month'] == mau_payment['log_month']), 'install', 'existing')
mau_payment.head()

In [ ]:
mau_payment_summary = mau_payment.groupby(['log_month', 'user_type'])['payment'].sum().reset_index()
mau_payment_summary.head()

数据可视化

In [ ]:
import matplotlib.pyplot as plt

plt.style.use('seaborn-deep')
plt.rcParams['font.sans-serif']=['SimHei'] #用来正常显示中文标签
plt.rcParams['axes.unicode_minus']=False #用来正常显示负号

%matplotlib inline

In [ ]:
width = 0.35
months = ['2013-06', '2013-07']
existing = [177886.0, 177886.0]
install = [49837.0, 29199.0]
p1 = plt.bar(months, existing, width)
p2 = plt.bar(months, install, width, bottom=existing)
plt.xlabel("log_month")
plt.ylabel("total payment")
plt.legend((p1[0], p2[0]), ('existing', 'install'))
plt.title("游戏销售额比较(上月/本月)")
plt.show()

In [ ]:
install = mau_payment.loc[(mau_payment['user_type'] == 'install') & (mau_payment['payment'] > 0)]
install.head()

In [ ]:
bins = np.linspace(0, 16000, 10)
month_06 = list(install.loc[install['log_month'] == '2013-06', 'payment'])
month_07 = list(install.loc[install['log_month'] == '2013-07', 'payment'])

In [ ]:
plt.hist([month_06, month_07], bins, label=['2013-06', '2013-07'])
plt.legend(loc='upper right')
plt.title("游戏新用户销售额比较(上月/本月)")
plt.show()